#  TESTE UBC

Nome: Roberto Souza

Passos feitos:  
1. Criar um Docker com Solr;  
2. Configurar o Solr;  
3. Tratamento Python
    a. Tratar Arquivo no Pandas;  
    b. Enviar um Arquivo para o Solr;  
    c. Pesquisar campos no Solr;  
    d. Adicionar um novo dado;  
    e. Editar um campo no Solr;  
    f. Excluir dado no Solr;  

9. Criar um Dashboard com os dados iniciais  

### 01 Criar um Docker com Solr

1. Instalar docker.
2. Criar uma instancia do Solr no Docker(prompt de comando)
   executar o comando abaixo:   
> docker run -d -p 8983:8983 --name solr_instance -t solr


### 02 Configurar o Solr

1. Criar um core no Solr
   executar o comando abaixo:
> docker create -c rcore

### 03 Tratamento Python

In [1]:
# Importando bibliotecas a serem utilizadas

import pysolr
import pandas as pd
import csv
import unicodedata
import os
import requests
 

In [2]:
# definindo diretorios e parametros
dir_file = 'E:/testeucb/'
dir_file_output = 'E:/testeucb/output/' # para onde vai o arquivo tratado
#os.mkdir(dir_file_output)


host_solr = 'http://localhost:8983/solr/'
name_core = 'rcore'

solr_url = 'http://localhost:8983/solr/'+name_core
print(solr_url)
#solr_url = 'http://localhost:8983/solr/rcore/select'

solr = pysolr.Solr(solr_url)

# SelectSolr
selectSolr = solr_url+'/select'

http://localhost:8983/solr/rcore


In [3]:
# Abrindo arquivo

df_aluno = pd.read_csv(dir_file+'aluno.csv', sep=',', encoding='utf-8')
df_aluno.isnull().sum()

Nome                  0
Idade                 0
Série                 0
Nota Média            0
Endereço              0
Nome do Pai           0
Nome da Mãe           0
Data de Nascimento    0
dtype: int64

In [4]:
# verificando nomes das colunas
df_aluno.columns.to_list()

['Nome',
 'Idade',
 'Série',
 'Nota Média',
 'Endereço',
 'Nome do Pai',
 'Nome da Mãe',
 'Data de Nascimento']

In [5]:
# Função para remover caracteres especiais e espaços, deixando em lowercase

def remover_acentos(texto):
    return ''.join((c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn'))


# Chamando a função e tratando as colunas
nomes_colunas = df_aluno.columns.to_list()
for nome in nomes_colunas:
    
    novo_nome = remover_acentos(nome.upper())
    novo_nome = novo_nome.replace(' ', '_')  
    df_aluno = df_aluno.rename(columns={nome : novo_nome})

In [6]:
# Verificando o dataset
df_aluno.head()

,NOME,IDADE,SERIE,NOTA_MEDIA,ENDERECO,NOME_DO_PAI,NOME_DA_MAE,DATA_DE_NASCIMENTO
0,Alice,10,5,8.5,123 Main St,John Doe,Jane Doe,2013-05-15
1,Bob,11,6,7.2,456 Oak St,Bob Smith,Alice Smith,2012-08-21
2,Charlie,9,4,9.0,789 Pine St,Charlie Brown,Lucy Brown,2014-02-10
3,David,10,5,8.8,101 Cedar St,David Johnson,Emily Johnson,2013-07-18
4,Emma,11,6,7.5,202 Elm St,Michael White,Sophia White,2012-10-05


In [7]:
# Verificando a quantidade
len(df_aluno)

78

In [8]:
# Gravando novo dataset
nome_arquivo = dir_file_output + 'n_aluno.csv'
df_aluno.to_csv(nome_arquivo, encoding = 'UTF-8', sep = ',', index=False, header=False)

In [9]:
#
#
#
#

In [10]:
#Abrindo arquivo salvo
df_naluno = pd.read_csv(nome_arquivo, sep=',', encoding='utf-8')
df_naluno.isnull().sum()

Alice          0
10             0
5              0
8.5            0
123 Main St    0
John Doe       0
Jane Doe       0
2013-05-15     0
dtype: int64

In [11]:
df_naluno#.head()

,Alice,10,5,8.5,123 Main St,John Doe,Jane Doe,2013-05-15
0,Bob,11,6,7.2,456 Oak St,Bob Smith,Alice Smith,2012-08-21
1,Charlie,9,4,9.0,789 Pine St,Charlie Brown,Lucy Brown,2014-02-10
2,David,10,5,8.8,101 Cedar St,David Johnson,Emily Johnson,2013-07-18
3,Emma,11,6,7.5,202 Elm St,Michael White,Sophia White,2012-10-05
4,Frank,9,4,9.2,303 Maple St,Frank Williams,Grace Williams,2014-01-22
...,...,...,...,...,...,...,...,...
72,Vincent,11,6,7.7,7171 Birch St,Vincent Harris,Lily Harris,2012-11-13
73,Wendy,9,4,9.6,7272 Oak St,Wendy Davis,Daniel Davis,2014-03-27
74,Xavier,10,5,8.2,7373 Pine St,Xavier Martin,Sophia Martin,2013-05-06
75,Yara,11,6,7.0,7474 Sycamore St,Yara Harris,John Harris,2012-10-18


In [12]:
#
# Enviando arquivo para o Solr
# obs: O solr tem que estar rodando

In [13]:
#df_aluno.columns.tolist()
t_nameColumns = pd.DataFrame(df_aluno.columns)
t_nameColumns = t_nameColumns.reset_index()
t_nameColumns = t_nameColumns.rename(columns={'index':'id', 0:'name'})
t_nameColumns

,id,name
0,0,NOME
1,1,IDADE
2,2,SERIE
3,3,NOTA_MEDIA
4,4,ENDERECO
5,5,NOME_DO_PAI
6,6,NOME_DA_MAE
7,7,DATA_DE_NASCIMENTO


In [14]:
# Quantidade de colunas
len(t_nameColumns)

8

In [15]:
# Tratamento na mão (Caso seja um novo arquivo)
# Copiar as tuplas abaixo, 
#     colocar ' (aspas nas colunas), 
#     retirar os espaços da numeração do ROW 
#     e acrescentar virgula no final (menos no ultimo)

i = 0
quant = len(t_nameColumns) 
while i < quant:  
    print(t_nameColumns['name'][i] , ': row[',t_nameColumns['id'][i],']')
    i+=1

NOME : row[ 0 ]
IDADE : row[ 1 ]
SERIE : row[ 2 ]
NOTA_MEDIA : row[ 3 ]
ENDERECO : row[ 4 ]
NOME_DO_PAI : row[ 5 ]
NOME_DA_MAE : row[ 6 ]
DATA_DE_NASCIMENTO : row[ 7 ]


In [16]:
#
# Enviando arquivo GERADO para o Solr
#
# Colar substituindo o trecho que fica dentro de tralhas abaixo ###

with open(nome_arquivo, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        doc = {###
            'NOME' : row[0],
            'IDADE' : row[1],
            'SERIE' : row[2],
            'NOTA_MEDIA' : row[3],
            'ENDERECO' : row[4],
            'NOME_DO_PAI' : row[5],
            'NOME_DA_MAE' : row[6],
            'DATA_DE_NASCIMENTO' : row[7]
        } ###
        solr.add([doc])

In [17]:
# Lendo todos os dados enviados para o Solr
df_solr = pd.DataFrame()
df_read = pd.DataFrame()
results = solr.search('*:*')
for result in results:
    df_read = pd.DataFrame(result)
    df_solr = pd.concat([df_solr,df_read])

df_solr = df_solr.reset_index()
df_solr = df_solr.drop(columns='index')

In [18]:
# DataFrame do aquivo lido no Solr
# Pega somente os 10 primeiros
df_solr

,NOME,IDADE,SERIE,NOTA_MEDIA,ENDERECO,NOME_DO_PAI,NOME_DA_MAE,DATA_DE_NASCIMENTO,id,_version_
0,Alice,10,5,8.5,123 Main St,John Doe,Jane Doe,2013-05-15T00:00:00Z,f418302e-ddd8-4459-8ff7-6e37eec877f9,1789814312710176768
1,Bob,11,6,7.2,456 Oak St,Bob Smith,Alice Smith,2012-08-21T00:00:00Z,9bfcc15a-be64-433b-a47b-d8cc5e8cccf6,1789814312718565376
2,Charlie,9,4,9.0,789 Pine St,Charlie Brown,Lucy Brown,2014-02-10T00:00:00Z,86e52738-96d3-49c8-a6e1-5df5e04ba181,1789814312765751296
3,David,10,5,8.8,101 Cedar St,David Johnson,Emily Johnson,2013-07-18T00:00:00Z,191313fd-615a-423e-a18a-6ced3c651f4d,1789814312818180096
4,Emma,11,6,7.5,202 Elm St,Michael White,Sophia White,2012-10-05T00:00:00Z,d25ca487-47e6-49e9-a230-472dcc1ab9c5,1789814312881094656
5,Frank,9,4,9.2,303 Maple St,Frank Williams,Grace Williams,2014-01-22T00:00:00Z,de5050f2-ccc1-42fa-9dfc-c848e7827cfe,1789814312933523456
6,Grace,10,5,8.0,404 Birch St,George Taylor,Olivia Taylor,2013-04-30T00:00:00Z,2f96e1ea-7bfb-495a-b931-d38f4a0edf17,1789814312985952256
7,Henry,11,6,7.8,505 Spruce St,Henry Moore,Lily Moore,2012-09-14T00:00:00Z,e80ae7df-a461-4311-84f0-955d0f442413,1789814313038381056
8,Isabel,9,4,9.5,606 Walnut St,Isaac Davis,Ava Davis,2014-03-07T00:00:00Z,95e681ac-3f4b-4aff-8fcb-a49991e39dd3,1789814313090809856
9,Jack,10,5,7.9,707 Sycamore St,Jack Smith,Emma Smith,2013-06-19T00:00:00Z,8452dca4-8440-48a4-b9b3-952f36217832,1789814313143238656


In [19]:
def buscar_dados(dig_option, fil):
    df_solr = pd.DataFrame()
    
    
    if dig_option == 1:
        fq = 'IDADE' 

    elif dig_option == 2:
        fq = 'NOME' 

    else:
        print('tenta uma das opções')
        print('Digite 1 para pesquisar por IDADE ')
        print('Digite 2 para pesquisar por NOME ')
    
    

    params = {
        'q': '*:*',  
        'fq': fq+': '+fil
    }

    # Realiza a busca no Solr
    response = requests.get(selectSolr, params=params)

    # Verifica se a requisição foi bem-sucedida (código 200)
    if response.status_code == 200: 
        search_results = response.json() 
 
        for doc in search_results['response']['docs']: 
            df_solr = pd.concat([df_solr,pd.DataFrame(doc)])
    else:
        print(f"Falha na requisição. Código de status: {response.status_code}")
        print(response.text)


    df_solr = df_solr.reset_index()
    df_solr = df_solr.drop(columns='index')

    return df_solr

In [20]:
print('Preencha o campo abaixo para pesquisar: ')

print('Digite 1 para pesquisar por IDADE ')
print('Digite 2 para pesquisar por NOME ')

dig_option= int(input('Digite o numero: '))

if dig_option == 1:
    dig_df = input('Digite a idade a')
    
elif dig_option == 2:
    dig_df = input('Digite nome a pesquisar: ')

else:
    print('tenta uma das opções')
    print('Digite 1 para pesquisar por IDADE ')
    print('Digite 2 para pesquisar por NOME ')
    
df_search = buscar_dados(dig_option,dig_df) 
df_search

Preencha o campo abaixo para pesquisar: 
Digite 1 para pesquisar por IDADE 
Digite 2 para pesquisar por NOME 
Digite o numero: 1
Digite a idade a10


,NOME,IDADE,SERIE,NOTA_MEDIA,ENDERECO,NOME_DO_PAI,NOME_DA_MAE,DATA_DE_NASCIMENTO,id,_version_
0,Alice,10,5,8.5,123 Main St,John Doe,Jane Doe,2013-05-15T00:00:00Z,f418302e-ddd8-4459-8ff7-6e37eec877f9,1789814312710176768
1,David,10,5,8.8,101 Cedar St,David Johnson,Emily Johnson,2013-07-18T00:00:00Z,191313fd-615a-423e-a18a-6ced3c651f4d,1789814312818180096
2,Grace,10,5,8.0,404 Birch St,George Taylor,Olivia Taylor,2013-04-30T00:00:00Z,2f96e1ea-7bfb-495a-b931-d38f4a0edf17,1789814312985952256
3,Jack,10,5,7.9,707 Sycamore St,Jack Smith,Emma Smith,2013-06-19T00:00:00Z,8452dca4-8440-48a4-b9b3-952f36217832,1789814313143238656
4,Mia,10,5,8.7,1010 Maple St,Ryan Brown,Mia Brown,2013-05-12T00:00:00Z,26381552-b3c5-4123-906c-b73d1ce665a0,1789814313300525056
5,Peter,10,5,8.4,1313 Elm St,Peter Clark,Ava Clark,2013-04-18T00:00:00Z,e215cdb6-00cf-464b-87e3-517c28d81f0a,1789814313457811456
6,Sam,10,5,8.6,1616 Sycamore St,Sam Turner,Emily Turner,2013-06-06T00:00:00Z,13a5d93e-2a0b-47fe-9cb1-8000edbfc92c,1789814313625583616
7,Victor,10,5,8.2,1919 Oak St,Victor Martin,Ella Martin,2013-05-24T00:00:00Z,aa00b6bd-db85-4d3e-9646-4fbda2f0b9d8,1789814313793355776
8,Yara,10,5,8.1,2222 Elm St,Yara Davis,John Davis,2013-04-04T00:00:00Z,6649fa35-5332-434c-8100-3764f0b7b6ba,1789814313961127936
9,Bella,10,5,8.9,2525 Sycamore St,Bella Martin,Ella Martin,2013-06-14T00:00:00Z,dc9fc9cb-125b-4164-b0d1-b3cbbe480e33,1789814314128900096


In [21]:
# Quantidade do Resultado
len(df_solr)

10

In [22]:
# Adicionando dados manualmente

print('Preencha os campos abaixos para ADICIONAR um novo registro')
print()

dig_nname= input('Nome: ')
dig_idade= int(input('Idade: '))
dig_serie= int(input('Serie: '))
dig_nota= float(input('NOTA_MEDIA: '))
dig_end= input('ENDERECO: ')
dig_pai= input('NOME_DO_PAI: ')
dig_mae= input('NOME_DA_MAE: ')
dig_nasc= input('DATA_DE_NASCIMENTO: ')

#doc = {'Name': 'Roberto', 'Idade': '32'}

doc = {###
    'NOME' : dig_nname,
    'IDADE' : dig_idade,
    'SERIE' : dig_serie,
    'NOTA_MEDIA' : dig_nota,
    'ENDERECO' : dig_end,
    'NOME_DO_PAI' : dig_pai,
    'NOME_DA_MAE' : dig_mae,
    'DATA_DE_NASCIMENTO' : dig_nasc
        } ###
solr.add([doc]) 

Preencha os campos abaixos para ADICIONAR um novo registro

Nome: Roberto
Idade: 32
Serie: 9
NOTA_MEDIA: 99.8
ENDERECO: Rua Z
NOME_DO_PAI: Jose
NOME_DA_MAE: Rita
DATA_DE_NASCIMENTO: 1991-12-14


'{\n  "responseHeader":{\n    "status":0,\n    "QTime":52\n  }\n}'

In [23]:
novoCampo = buscar_dados(2,dig_nname)
novoCampo

,NOME,IDADE,SERIE,NOTA_MEDIA,ENDERECO,NOME_DO_PAI,NOME_DA_MAE,DATA_DE_NASCIMENTO,id,_version_
0,Roberto,32,9,99.8,Rua Z,Jose,Rita,1991-12-14T00:00:00Z,d686cdb1-07c9-4d9f-a066-7caeb215b77a,1789814357235859456


In [24]:
#
# SOMENTE DESCOMENTAR SE FOR REALMENTE EXCLUIR
#

# Deletar somente um campo
# buscar o id e substituir
#solr.delete(id='0e972e54-041e-410c-88e4-5d6168d38eb8')

# Deletar todos os campos
#solr.delete(q='*:*')


In [26]:
# Alterando um campo
# Atenção no ID

doc = {'id': 'd686cdb1-07c9-4d9f-a066-7caeb215b77a', 
    'NOME' : 'Roberto Souza',
    'IDADE' : 32,
    'SERIE' : 5,
    'NOTA_MEDIA' : 10,
    'ENDERECO' : 'Rua Za',
    'NOME_DO_PAI' : 'Jose',
    'NOME_DA_MAE' : 'Rita',
    'DATA_DE_NASCIMENTO' : '1991-12-14'
      }
solr.add([doc], overwrite=True)

novoCampo = buscar_dados(2,'Roberto Souza')
novoCampo

,NOME,IDADE,SERIE,NOTA_MEDIA,ENDERECO,NOME_DO_PAI,NOME_DA_MAE,DATA_DE_NASCIMENTO,id,_version_
0,Roberto,32,9,99.8,Rua Z,Jose,Rita,1991-12-14T00:00:00Z,d686cdb1-07c9-4d9f-a066-7caeb215b77a,1789814357235859456
1,Roberto Souza,32,5,10.0,Rua Za,Jose,Rita,1991-12-14T00:00:00Z,9b57e396-b8d6-4fdd-8bc2-18a3a46d895f,1789814379202478080


In [28]:
novoCampo = buscar_dados(2,'Roberto Souza')
novoCampo

,id,NOME,IDADE,SERIE,NOTA_MEDIA,ENDERECO,NOME_DO_PAI,NOME_DA_MAE,DATA_DE_NASCIMENTO,_version_
0,9b57e396-b8d6-4fdd-8bc2-18a3a46d895f,Roberto Souza,32,5,10.0,Rua Za,Jose,Rita,1991-12-14T00:00:00Z,1789814379202478080


In [ ]:
# Fim de Codigo